### Testy statystyczne

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import mannwhitneyu
from scipy.stats import shapiro

In [ ]:
df_prep = pd.read_csv("..//datasets/gasometry_prepared_data.csv")
cols=["BETET",	"CO2TET",	"HCO3ACTE",	"HCO3STTE",	"O2SATTET",	"O2TET", "IONH"]
my_blue="#0064B2"
my_red="#D61600"

In [ ]:
normality_df = pd.DataFrame(index=cols, columns=['Statystyka_0.0', 'P-wartość_0.0', 'Statystyka_1.0', 'P-wartość_1.0'])

# Podział danych względem klasy zgon
grupy = df_prep.groupby('ZGON')

# Dla każdej zmiennej przeprowadzenie testu normalności Shapiro-Wilka dla obu grup
for column in df_prep[cols]:  # Iteracja po wszystkich kolumnach oprócz ostatniej (zgon)
    for group, data_group in grupy:
        stat, p = shapiro(data_group[column])
        normality_df.at[column, f'Statystyka_{group}'] = stat
        normality_df.at[column, f'P-wartość_{group}'] = p

print(normality_df)
normality_df.to_excel("normality.xlsx")

In [ ]:
df_dead = df_prep[df_prep["ZGON"]==1].copy()
df_life = df_prep[df_prep["ZGON"]==0].copy()
tabela_MN=pd.DataFrame(columns=("Zmienna", "Statystyka U", "P-wartość_two","P-wartość_less", "P-wartość_greater"))
for column in cols:
    mw2 = mannwhitneyu(df_dead[column], df_life[column], alternative="two-sided")
    mw_less = mannwhitneyu(df_dead[column], df_life[column], alternative="less")
    mw_greater = mannwhitneyu(df_dead[column], df_life[column], alternative="greater")
    new_row = {"Zmienna": column, "Statystyka U": mw2[0], "P-wartość_two": mw2[1], "P-wartość_less": mw_less[1], "P-wartość_greater": mw_greater[1]}
    tabela_MN = pd.concat([tabela_MN, pd.DataFrame([new_row])], ignore_index=True)
# tabela_MN.to_excel("wyniki_MN.xlsx")
tabela_MN

In [ ]:
params=[*cols, "PHTET"]
df_line = df_prep[[*params, "ZGON", "BADANIE_NR"]].copy()
df_line["ZGON"].replace({0: "nie", 1: "tak"}, inplace=True)

sns.set(font_scale=1.5)
sns.set_style("whitegrid")

# Rysowanie wykresów dla każdego parametru
for i, param in enumerate(params):
    # Tworzenie nowego obrazu dla każdego parametru
    plt.figure(figsize=(10, 7.5), facecolor='white')

    plt.axhline(y=0, linestyle='--', color="black")
    plt.axhline(y=1, linestyle='--', color="black")
    
    sns.lineplot(data=df_line, x="BADANIE_NR", y=param, hue="ZGON", ci='sd',
                 palette={'nie': my_blue, 'tak': my_red})

    plt.xlabel('Pomiar')
    plt.ylabel(f'{param}')
    plt.xlim(0, 50)
    plt.xticks(ticks=[1, 10, 20, 30, 40, 50])
    plt.legend(title="Zgon")
    plt.tight_layout()

    # Zapisywanie obrazu do pliku o nazwie "line_plot_param.png"
    plt.savefig(f'images3/plot_in_time_{i}.png')

    # Pokazywanie obrazu
    plt.show()

In [ ]:
df_prep = pd.read_csv("../chapter3/gasometry_prepared_data.csv")

In [ ]:
df_prep.drop(columns=["Unnamed: 0"], inplace=True)
df_zgon_0 = df_prep[df_prep['ZGON'] == 0]
df_zgon_1 = df_prep[df_prep['ZGON'] == 1]

# # Stwórz dwa wykresy
fig, axes = plt.subplots(1, 2, figsize=(20, 8), sharey=True)

# Wykres dla ZGON=0
for pacjent_nr, group in df_zgon_0.groupby('PACJENT_NR'):
        axes[0].plot(group['BADANIE_NR'].values, group['IONH'].values, label=f'Pacjent {int(pacjent_nr)}')

axes[0].set_title('ZGON=NIE', fontsize=16)
axes[0].set_xlabel('Numer badania', fontsize=14)
axes[0].set_ylabel('Pomiar IONH', fontsize=14)
axes[0].tick_params(axis='both', which='major', labelsize=12)

# # Wykres dla ZGON=1
for pacjent_nr, group in df_zgon_1.groupby('PACJENT_NR'):
        axes[1].plot(group['BADANIE_NR'].values, group['IONH'].values, label=f'Pacjent {int(pacjent_nr)}')

axes[1].set_title('ZGON=TAK', fontsize=16)
axes[1].set_xlabel('Numer badania', fontsize=14)
axes[1].set_ylabel('Pomiar IONH', fontsize=14)
axes[1].tick_params(axis='both', which='major', labelsize=12)

plt.tight_layout()
plt.savefig('images3/porownanie_szeregi_IONH.png')
plt.show()